In [121]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import json

agent_labels = {'BayesAgent':(r'AI$\xi$','red'),
          'MC-AIXI':('MC-AIXI','red'),
          'MC-AIMU':('MC-AIMU','blue'),
          'MDL Agent':('MDL','blue'),
          'MC-AIXI-Dirichlet':('MC-AIXI-Dirichlet','blue'),
          'Knowledge-seeking agent':('Kullback-Leibler','blue'),
          'KullbackLeiblerKSA':('Kullback-Leibler','blue'),
          'ShannonKSA':('Shannon','green'),
          'SquareKSA':('Square','red'),
          'Shannon KSA':('Shannon','orange'),
          'Square KSA':('Square','red'),
          'ThompsonAgent':('Thompson Sampling','blue'),
          'Thompson Sampling':('Thompson Sampling','blue'),
          'QLearn':('Q-Learning','black'),
          'KSA-Dirichlet': ('Kullback-Leibler','blue'),
          'Entropy-seeking agent': ('Shannon','orange'),
          'Square KSA-Dirichlet': ('Square','red')}


def plot_results(directory,
                 filename='results-1',
                 objective=None,
                 outfile=None,
                 show_optimal=False,
                 show_variance=True):
    if not objective:
        if 'ksa' in directory:
            objective = 'explored'
        else:
            objective = 'rewards'
    if objective == 'rewards':
        y_axis = 'Average Reward'
    elif objective == 'explored':
        y_axis = 'Exploration (%)'
    
    file = open(directory + '/' + filename + '.json')
    data = json.load(file)
    file.close()

    fig = plt.figure(figsize=(12,8),dpi=200)
    for i,k in enumerate(data):
        try:
            d = data[k][:50]
        except KeyError:
            continue

        cycles = d[0]['cycles']
        runs = len(d)

        A = np.zeros((cycles,runs))
        for j in range(runs):
            A[:,j] = np.array(d[j][objective])
        mu = np.mean(A,1)
        sigma = np.std(A,1)
        a = np.max(np.vstack((mu-sigma,np.min(A,1))),0)
        b = np.min(np.vstack((mu+sigma,np.array(cycles*[100]))),0)

        if k in agent_labels:
            lab = agent_labels[k][0]

        color = agent_labels[k][1]
        alpha = 0.1
        if show_variance:
            plt.plot(a,color=color,alpha=alpha)
            plt.plot(b,color=color,alpha=alpha)
            plt.fill_between(np.arange(cycles),a,b,alpha=alpha,color=color)
        
        plt.plot(mu,label=lab,color=color,lw=3)
        

    if objective=='rewards' and show_optimal:
        xs = np.array(range(cycles))
        ys = np.zeros(cycles)
        ys[:11] = -1.
        ys[11:] = 75.
        ys = np.cumsum(ys)

        ys[1:] /= xs[1:]
        plt.plot(xs,ys,'k--',lw=3,label='Optimal')
        
    plt.xlabel('Cycles',fontsize=25)
    plt.ylabel(y_axis,fontsize=25)
    plt.legend(fontsize=25)
    
    if outfile:
        plt.savefig(directory + '/' + outfile + '.png', bbox_inches='tight')
        plt.close()
        
plot_results('ksa','results-2')